# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Susuman,RU,62.7833,148.1667,48.67,75,75,1.92,broken clouds
1,1,Adamstown,PN,-25.0660,-130.1015,65.91,67,67,22.26,broken clouds
2,2,Tiksi,RU,71.6872,128.8694,63.70,49,10,6.82,clear sky
3,3,Namibe,AO,-15.1961,12.1522,63.23,88,66,7.52,broken clouds
4,4,Utrik,MH,11.2278,169.8474,82.89,72,86,11.30,light rain


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

ideal_weather=city_data_df.loc[city_data_df["Max Temp"] <=27] 
ideal_weather=city_data_df.loc[city_data_df["Max Temp"] >= 21]                                
ideal_weather=city_data_df.loc[city_data_df["Wind Speed"] < 4.5]
ideal_weather=city_data_df.loc[city_data_df["Wind Speed"] < 4.5]
ideal_weather=city_data_df.loc[city_data_df["Cloudiness"] == 0]                                                            

# Drop any rows with null values
ideal_weather_df=ideal_weather.dropna()

# Display sample data

ideal_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
14,14,Carloforte,IT,39.1450,8.3058,71.26,81,0,9.93,clear sky
32,32,Tuscaloosa,US,33.2098,-87.5692,91.18,66,0,6.91,clear sky
42,42,Umba,RU,66.6871,34.3429,57.18,79,0,6.38,clear sky
46,46,Port Elizabeth,ZA,-33.9180,25.5701,62.89,77,0,17.27,clear sky
56,56,Keflavik,IS,64.0049,-22.5624,57.11,71,0,6.91,clear sky


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

/var/folders/w_/68s58y2x4rzgbj4xhc12pms00000gn/T/ipykernel_3174/3351056220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
14,Carloforte,IT,39.1450,8.3058,81,
32,Tuscaloosa,US,33.2098,-87.5692,66,
42,Umba,RU,66.6871,34.3429,79,
46,Port Elizabeth,ZA,-33.9180,25.5701,77,
56,Keflavik,IS,64.0049,-22.5624,71,
...,...,...,...,...,...,...
470,Toliara,MG,-23.3500,43.6667,82,
495,Khandyga,RU,62.6667,135.6000,52,
504,Ciudad Valles,MX,21.9833,-99.0167,25,
522,Golbasi,TR,37.7836,37.6367,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": 'accommodation.hotel',
          "limit": 1,
          "apiKey": geoapify_key 
          
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Carloforte - nearest hotel: Hotel Riviera
Tuscaloosa - nearest hotel: Embassy Suites Tuscaloosa
Umba - nearest hotel: Гостиница
Port Elizabeth - nearest hotel: Waterford Hotel
Keflavik - nearest hotel: Núpan Deluxe
Ushtobe - nearest hotel: No hotel found
Bethel - nearest hotel: Hampton Inn Danbury
Al Jawf - nearest hotel: No hotel found
Behompy - nearest hotel: No hotel found
Tromso - nearest hotel: Clarion Collection Hotel Aurora
Gadzhiyevo - nearest hotel: Оазис
Port Mathurin - nearest hotel: Escale Vacances
Sayat - nearest hotel: No hotel found
Colchani - nearest hotel: Hotel de Sal - Sol de Mañana
Manghit - nearest hotel: Mang'it miymanxanasi
Tupiza - nearest hotel: Aramayo Hostal
Chissamba - nearest hotel: No hotel found
Tralee - nearest hotel: The Ashe Hotel
Tarija - nearest hotel: Hostal Bolivar
Cukurca - nearest hotel: No hotel found
Hermanus - nearest hotel: Aloe guest house
Hafar Al-Batin - nearest hotel: فندق الفرحان
Longyearbyen - nearest hotel: The Va

,City,Country,Lat,Lng,Humidity,Hotel Name
14,Carloforte,IT,39.1450,8.3058,81,Hotel Riviera
32,Tuscaloosa,US,33.2098,-87.5692,66,Embassy Suites Tuscaloosa
42,Umba,RU,66.6871,34.3429,79,Гостиница
46,Port Elizabeth,ZA,-33.9180,25.5701,77,Waterford Hotel
56,Keflavik,IS,64.0049,-22.5624,71,Núpan Deluxe
...,...,...,...,...,...,...
470,Toliara,MG,-23.3500,43.6667,82,Ambary
495,Khandyga,RU,62.6667,135.6000,52,Апельсин
504,Ciudad Valles,MX,21.9833,-99.0167,25,Hotel Riviera
522,Golbasi,TR,37.7836,37.6367,34,Güney Palas Otel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5,
    hover = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)